# Automated feature selection with boruta #
[Source on Kaggle](:https://www.kaggle.com/code/scratchpad/notebook82489a1e12/edit) by A. Bilogur: https://www.residentmar.io/)
## Introduction ##
In a previous notebook, ["Automated feature selection with sklearn"](https://www.kaggle.com/code/residentmario/automated-feature-selection-with-sklearn/notebook "Notebook on Kaggle"), I introduced the motivation and a sequence of sklearn solutions for the problem of feature selection.

Boruta is an R algorithm, since ported to Python, which implements another approach to feature selection. It's both significantly more complex than the algorithms offered in sklearn, and also carries itself a bit differently in terms of what its goal is. In introducing it I'll pull from the paper that was published alongside the algorithm (Google "Boruta" to find it).

## Background ##
Minimal-optimal feature selection versus all-relevant feature selection
Basically boruta is an algorithm designed to sovle what the paper calls the "all-relevant problem": finding a subset of features from the dataset which are relevant to given classification task. This is different from the "minimal-optimal problem", which is the problem of finding the minimal subset of features which are performant in a model. While machine learning models in production should ultimately target building on minimal-optimal selections of the data, the thesis of Boruta is that for the purposes of exploration minimum-optimality goes too far.

Boruta attempts to curate features down to the "all-relevant" stopping point, not the "minimal-optimal" stopping point. What does "all-relevant" mean? The paper defines a variable as being relevant if there is a subset of attributes in the dataset among which the variable is not redundant when used for prediction. To illustrate this, suppose that we have two variables, A and B. These variables are correlated with one another, and correlated with the objective variable. However, A is a stronger signal than B is. In a regular decision tree the A feature will be given high importance whle the B feature will be mostly ignored in A's favor. If we were minimizing our variable count then we would remove B.

## Decision trees, random forests, and gini inefficiency ##
The core algorithm behind boruta is random forests. Random forests are themselves based on decision trees, so I need to briefly discuss those first (more details on this algorithm in this notebook). A decision tree is a sequence of steps (decisions, or splits) that are calcuated at training time. A simple decision tree might classify every point that has X < 0.5 to class 0, and every point that has X > 0.5 to class 1. Each new data point flows down the tree, either down the left branch or the right branch, and ultimately arrives at its classification result.

In practice decision trees can subject points to dozens of tests as they "flow" down the tree before assigning their ultimate classification, and if you allow arbitrarily many splits, you can generate a perfectly overfitted decision tree! To control for this we may specify that we will only continue to split nodes so long as the decision results in a certain minimum benefit. The definition of benefit that is usually used is reduction in Gini impurity. Gini impurity is an only slightly complicated measurement related to the Gini coefficient (and the subject of a future notebook) that essentially measures how well the classifier is performing on a given subset of data.

Gini impurity has the important property that it, like decision trees, is non-parametric, and hence (for a large enough sample size) we can expect it to work with data in any numerical format. This is the reason that decision trees in sklearn are able to return a feature_importance_ attribute, and the reason that said attribute is so useful for choosing your input variables.

A random forest meanwhile is an ensemble of weak decision trees. In a random forest you train hundreds of purposefully overfitted decision trees, with each decision tree only gaining access to a random subset of the columns in the dataset. To classify an incoming point you have each of these trees cast a vote as to where to put it, and the majority vote wins.

The key insight in random forests, and the reason that they perform better than decision trees alone, is mass voting. By increasing the randomness of the decision trees being built we naturally increase their bias, but by then averaging their decisions we naturally reduce the variance. If a random forest is able to decrease variance more than it increases bias, relative to a single well-pruned decision tree, then it will perform better as a classifier on the dataset (albeit one that takes at least an order of magnitude longer to train).

## The algorithm ##
Boruta works by taking this randomness even further. Here's the algorithm:

1. Extend the information system by adding copies of all variables (the information system is always extended by at least 5 shadow attributes, even if the number of attributes in the original set is lower than 5).
2. Shuffle the added attributes to remove their correlations with the response.
3. Run a random forest classifier on the extended information system and gather the Z scores computed.
4. Find the maximum Z score among shadow attributes (MZSA), and then assign a hit to every attribute that scored better than MZSA.
5. For each attribute with undetermined importance perform a two-sided test of equality with the MZSA.
6. Deem the attributes which have importance significantly lower than MZSA as ‘unimportant’ and permanently remove them from the information system.
7. Deem the attributes which have importance significantly higher than MZSA as ‘important’.
8. Remove all shadow attributes.
9. Repeat the procedure until the importance is assigned for all the attributes, or the algorithm has reached the previously set limit of the random forest runs.

What basically happens is that randomly shuffled shadow attributes are defined to establish a baseline performance for prediction against the target variable. Then a hypothesis test is used to determine, with a certain level of risk (0.05 by default), if each variable is correlated only randomly. Variables that fail to be rejected are removed from consideration.

Recall that we had a feature B that was redundant to feature A, but still related to the objective variable. By training a random forest boruta will, by sheer chance, generate decision trees having the B feature but lacking the A feature. In these cases, in the absence of feature A, feature B will be given high importance instead. This importance will bouy the average importance assigned to that variable and prevent it from failing the test and being removed.

Obviously the larger the feature space and the more variables B is redundant to, the larger the number of trees that must be in the forest for a tree where B is not redudant to occur. However, if you use a forest with a parameterization that has been cross-validated to be optimal, you should be fine.

Simultaneously, as more and more uninformative variables are removed, the feature importance of the remaining informative variables will improve. Noisier (more random) related variables will see larger gains, as they will be decorrelated from the noise being pruned from the dataset. To illustrate this the boruta paper includes the following visualization showing variable pruning on a synthetic dataset:

In this example there is one noise green variable which is lifted very far up from the random noise in terms of Z-score as the algorithm iterates along. This effect is why boruta needs to run dozens of iterations to be effective. In this synthetic case boruta prunes down from 500 to 20 variables.

## Demonstration ##
Here's a sample demonstration on the same kepler dataset used for the sklearn tests in the previous notebook.

In [2]:
import pandas as pd
import numpy as np

In [5]:
#pd.set_option('max_columns', None)
kepler = pd.read_csv("kepler_cumulative.csv")
#kepler = pd.read_csv("../input/cumulative.csv")
kepler = (kepler
     .drop(['rowid', 'kepid'], axis='columns')
     .rename(columns={'koi_disposition': 'disposition', 'koi_pdisposition': 'predisposition'})
     .pipe(lambda df: df.assign(disposition=(df.disposition == 'CONFIRMED').astype(int), predisposition=(df.predisposition == 'CANDIDATE').astype(int)))
     .pipe(lambda df: df.loc[:, df.dtypes.values != np.dtype('O')])  # drop str columns
     .pipe(lambda df: df.loc[:, (df.isnull().sum(axis='rows') < 500).where(lambda v: v).dropna().index.values])  # drop columns with greater than 500 null values
     .dropna()
)

kepler_X = kepler.iloc[:, 1:]
kepler_y = kepler.iloc[:, 0]

kepler.head()

,disposition,predisposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,1,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,1,1,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,0,0,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,0,0,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,1,1,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [7]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=200, n_jobs=-1, max_depth=5)

trans = BorutaPy(clf, random_state=42, verbose=2)
sel = trans.fit_transform(kepler_X.values, kepler_y.values)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	40
Tentative: 	1
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	40
Tentative: 	1
Rejected: 	0
Iteration: 	10 / 100
Confirmed: 	40
Tentative: 	1
Rejected: 	0
Iteration: 	11 / 100
Confirmed: 	40
Tentative: 	1
Rejected: 	0
Iteration: 	12 / 100
Confirmed: 	40
Tentative: 	1
Rejected: 	0
Iteration: 	13 / 100
Confirmed: 	40
Tentative: 	1
Rejected: 	0
Iteration: 	14 / 100
Confirmed: 	40
Tentative: 	1
Rejected: 	0
Iteration: 	15 / 100
Confirmed: 	40
Tentative: 	1
Rejected: 	0
Iteration: 	16 / 100
Confirmed: 	40
Tentative: 	1
Rejected: 	0
I

In [8]:
trans.ranking_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1])

According to boruta, every variable in this dataset is relevant to the target variable in at least a minimal way. In other words, with this Kepler dataset the solution to the all-relevant problem is all of the fields, and we shouldn't be rejecting anything! That's a really stark contrast to the results we got out of the sklearn functions, and really illustrates the difference between maximal-minimal and all-relevant quite well.

## Conclusion ##

boruta is a very interesting and very powerful feature selection algorithm which general applicability across almost all datasets. It's got one huge disadvantage however: even when used with lower-complexity trees, it still has an execution time measured in hours, not the seconds provided by all of the sklearn tools.

This fact also makes it very, very hard to tune, as every parameter tuning will require extremely high additional CPU time. So you should only use it when this execution wait time is "worth it". In particular, I would suggest using it for particularly hairy datasets, one with hundreds of potentially weakly correlated predictor variables. In simpler cases, and in cases like the one in this notebook it doesn't seem worth it.

### For some other boruta demos see: ###
["Boruta Feature Elimination"](https://www.kaggle.com/code/tilii7/boruta-feature-elimination/notebook "Kaggle Notebook")
<br>
["Boruta Fearuter Importance Analysis"](https://www.kaggle.com/code/jimthompson/boruta-feature-importance-analysis/report "Kaggle Notebook")
